In [ ]:
import pandas as pd
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords,wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import pos_tag
from nltk.util import ngrams
from nltk.chunk import RegexpParser
import contractions
import re
import emoji
from typing import List, Tuple
from sklearn.model_selection import train_test_split
import os
import spacy
nlp = spacy.load("en_core_web_sm")

In [2]:
# Cleaning functions
def clean_text(text: str) -> str:
    """Clean raw text by removing URLs, emojis, mentions, and MBTI codes."""
    text = text.lower()
    text = re.sub(r'https?\S+|www\S+', '', text)
    text = emoji.replace_emoji(text, replace='')
    text = re.sub(r'@\w+|#', '', text)
    text = re.sub(r"[^a-z\']", ' ', text)
    # Remove MBTI type codes (e.g., INFJ, ENTP) to avoid leaking information
    text = re.sub(r'\b(I|E)(N|S)(F|T)(J|P)\b', '', text, flags=re.IGNORECASE)
    # Remove common footer
    text = re.sub(r'\bsent (from )?my \w+(\s\w+)? using tapatalk\b', '', text, flags=re.IGNORECASE)
    text = re.sub(r'w w w', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def preprocess_posts(posts_str: str) -> str:
    """Split multi-post string (separated by '|||'), clean each, and join."""
    posts = posts_str.split('|||')
    cleaned_posts = [clean_text(post) for post in posts]
    joined = ' '.join(cleaned_posts)
    return re.sub(r'\s+', ' ', joined).strip()

In [3]:
# Encoding function
def encode_mbti_type(mbti: str) -> Tuple[int, int, int, int]:
    """
    Encode MBTI type as 4 binary dimensions:
    I/E → 1/0, N/S → 1/0, F/T → 1/0, J/P → 1/0
    """
    return (
        1 if mbti[0] == 'I' else 0,
        1 if mbti[1] == 'N' else 0,
        1 if mbti[2] == 'F' else 0,
        1 if mbti[3] == 'J' else 0,
    )

In [4]:
# Initialize NLTK components
STOP_WORDS = set(stopwords.words('english'))
WNL = WordNetLemmatizer()
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

def get_wordnet_pos(tag: str) -> str:
    """Map POS tag to WordNet format for lemmatization."""
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mayasalkhateeb/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/mayasalkhateeb/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mayasalkhateeb/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/mayasalkhateeb/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/mayasalkhateeb/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
# Tokenization variants
def tokens_without_lemma(text: str) -> List[str]:
    fixed = contractions.fix(text)
    tokens = word_tokenize(fixed)
    filtered_tokens = [word for word in tokens if word.lower() not in STOP_WORDS]
    return filtered_tokens

def tokens_with_lemma(text: str) -> List[str]:
    fixed = contractions.fix(text)
    tokens = word_tokenize(fixed)
    filtered_tokens = [word for word in tokens if word.lower() not in STOP_WORDS]
    lemmatized = [WNL.lemmatize(word.lower()) for word in filtered_tokens]
    return lemmatized

def tokens_with_lemma_pos(text: str) -> List[str]:
    fixed = contractions.fix(text)
    tokens = word_tokenize(fixed)
    filtered_tokens = [word for word in tokens if word.lower() not in STOP_WORDS]
    pos_tags = pos_tag(filtered_tokens)
    lemmatized = [
        WNL.lemmatize(token.lower(), pos=get_wordnet_pos(pos))
        for token, pos in pos_tags
    ]
    return lemmatized

def tokens_with_spacy_dep(text: str) -> List[str]:
    doc = nlp(contractions.fix(text))
    relations = []
    for sent in doc.sents:
        for token in sent:
            if token.pos_ == 'VERB':
                subj = next((child.text for child in token.children if child.dep_ == 'nsubj'), None)
                obj = next((child.text for child in token.children if child.dep_ in ('dobj', 'pobj')), None)
                if subj and obj:
                    relations.append(f"{subj}-{token.lemma_}-{obj}")
    return relations

def tokens_with_spacy_chunking(text: str) -> List[str]:
    doc = nlp(contractions.fix(text))
    chunks = [chunk.text for chunk in doc.noun_chunks] + [vp.text for vp in doc if vp.pos_ == 'VERB']
    return chunks

In [6]:
# N-gram generation
def generate_ngrams(tokens: List[str]) -> Tuple[List[Tuple], List[Tuple], List[Tuple]]:
    if not tokens:
        return [], [], []
    unigrams = list(ngrams(tokens, 1))
    bigrams = list(ngrams(tokens, 2))
    trigrams = list(ngrams(tokens, 3))
    return unigrams, bigrams, trigrams

In [7]:
# Main processing
df = pd.read_csv("../data/raw/mbti_1.csv")  # Adjust path if needed

print("🧹 Cleaning posts...")
tqdm.pandas()
df['cleaned_posts'] = df['posts'].progress_apply(preprocess_posts)

print("🏷️ Encoding MBTI types...")
df[['IE', 'NS', 'FT', 'JP']] = pd.DataFrame(
    df['type'].progress_apply(encode_mbti_type).tolist(),
    index=df.index
)

variants = [
    'without_lemma',
    'with_lemma',
    'with_lemma_pos',
    'with_dep_tree',
    'with_chunking'
]

for var in variants:
    print(f"🔤 Processing variant: {var}")
    if var == 'without_lemma':
        func = tokens_without_lemma
    elif var == 'with_lemma':
        func = tokens_with_lemma
    elif var == 'with_lemma_pos':
        func = tokens_with_lemma_pos
    elif var == 'with_dep_tree':
        func = tokens_with_dependency_tree
    elif var == 'with_chunking':
        func = tokens_with_chunking
    
    df[f'tokens_{var}'] = df['cleaned_posts'].progress_apply(func)
    
    print(f"📊 Generating n-grams for {var}...")
    ngram_results = df[f'tokens_{var}'].progress_apply(generate_ngrams)
    df[[f'Unigrams_{var}', f'Bigrams_{var}', f'Trigrams_{var}']] = pd.DataFrame(
        ngram_results.tolist(), index=df.index
    )

# Drop intermediate columns
df = df.drop(columns=['posts', 'cleaned_posts'])



🧹 Cleaning posts...


100%|██████████| 8675/8675 [00:57<00:00, 150.83it/s]


🏷️ Encoding MBTI types...


100%|██████████| 8675/8675 [00:00<00:00, 1114583.77it/s]


🔤 Processing variant: without_lemma


...

100%|██████████| 8675/8675 [00:04<00:00, 1819.13it/s]


🔤 Processing variant: with_chunking


100%|██████████| 8675/8675 [07:21<00:00, 19.66it/s]


📊 Generating n-grams for with_chunking...


100%|██████████| 8675/8675 [00:02<00:00, 3941.47it/s]


In [9]:
# Split into train and test
print("✂️ Splitting data into train/test...")
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['type'])

# Save to ../data/processed
os.makedirs('../data/processed', exist_ok=True)
train_df.to_pickle('../data/processed/train.pkl')
test_df.to_pickle('../data/processed/test.pkl')

print("✅ Processing complete! Data saved to ../data/processed/train.pkl and test.pkl")

✂️ Splitting data into train/test...
✅ Processing complete! Data saved to ../data/processed/train.pkl and test.pkl


In [10]:
train_df.head()

,type,IE,NS,FT,JP,tokens_without_lemma,Unigrams_without_lemma,Bigrams_without_lemma,Trigrams_without_lemma,tokens_with_lemma,...,Bigrams_with_lemma_pos,Trigrams_with_lemma_pos,tokens_with_dep_tree,Unigrams_with_dep_tree,Bigrams_with_dep_tree,Trigrams_with_dep_tree,tokens_with_chunking,Unigrams_with_chunking,Bigrams_with_chunking,Trigrams_with_chunking
8331,INFP,1,1,1,0,"['this, actually, exactly, expected, laughing,...","[('this,), (actually,), (exactly,), (expected,...","[('this, actually), (actually, exactly), (exac...","[('this, actually, exactly), (actually, exactl...","['this, actually, exactly, expected, laughing,...",...,"[('this, actually), (actually, exactly), (exac...","[('this, actually, exactly), (actually, exactl...","['this-is-i, i-expected-introversion, i-laughi...","[('this-is-i,), (i-expected-introversion,), (i...","[('this-is-i, i-expected-introversion), (i-exp...","[('this-is-i, i-expected-introversion, i-laugh...","['this, is, i, expected, laughing introversion...","[('this,), (is,), (i,), (expected,), (laughing...","[('this, is), (is, i), (i, expected), (expecte...","[('this, is, i), (is, i, expected), (i, expect..."
1290,ISTP,1,0,0,0,"['nope, ever, busy, work, causes, adrenaline, ...","[('nope,), (ever,), (busy,), (work,), (causes,...","[('nope, ever), (ever, busy), (busy, work), (w...","[('nope, ever, busy), (ever, busy, work), (bus...","['nope, ever, busy, work, cause, adrenaline, r...",...,"[('nope, ever), (ever, busy), (busy, work), (w...","[('nope, ever, busy), (ever, busy, work), (bus...","[i-do-anything, anything-make-one, one-keep-fr...","[(i-do-anything,), (anything-make-one,), (one-...","[(i-do-anything, anything-make-one), (anything...","[(i-do-anything, anything-make-one, one-keep-f...","[i, am, busy, with work causes, work causes, a...","[(i,), (am,), (busy,), (with work causes,), (w...","[(i, am), (am, busy), (busy, with work causes)...","[(i, am, busy), (am, busy, with work causes), ..."
1982,ENFJ,0,1,1,1,"['yes, peace, absence, conflict, friend, suxx,...","[('yes,), (peace,), (absence,), (conflict,), (...","[('yes, peace), (peace, absence), (absence, co...","[('yes, peace, absence), (peace, absence, conf...","['yes, peace, absence, conflict, friend, suxx,...",...,"[('yes, peace), (peace, absence), (absence, co...","[('yes, peace, absence), (peace, absence, conf...","[peace-is-absence, suxx-hardd-peace, peace-be-...","[(peace-is-absence,), (suxx-hardd-peace,), (pe...","[(peace-is-absence, suxx-hardd-peace), (suxx-h...","[(peace-is-absence, suxx-hardd-peace, peace-be...","['yes peace, is the absence of conflict, the a...","[('yes peace,), (is the absence of conflict,),...","[('yes peace, is the absence of conflict), (is...","[('yes peace, is the absence of conflict, the ..."
769,INFP,1,1,1,0,"[', apologize, delayed, response, thank, takin...","[(',), (apologize,), (delayed,), (response,), ...","[(', apologize), (apologize, delayed), (delaye...","[(', apologize, delayed), (apologize, delayed,...","[', apologize, delayed, response, thank, takin...",...,"[(', apologize), (apologize, delay), (delay, r...","[(', apologize, delay), (apologize, delay, res...","[i-apologize-response, you-taking-time, it-is-...","[(i-apologize-response,), (you-taking-time,), ...","[(i-apologize-response, you-taking-time), (you...","[(i-apologize-response, you-taking-time, it-is...","[i, apologize for the delayed response, for th...","[(i,), (apologize for the delayed response,), ...","[(i, apologize for the delayed response), (apo...","[(i, apologize for the delayed response, for t..."
8339,INFP,1,1,1,0,"['nightglow, even, imagine, must, struggling, ...","[('nightglow,), (even,), (imagine,), (must,), ...","[('nightglow, even), (even, imagine), (imagine...","[('nightglow, even, imagine), (even, imagine, ...","['nightglow, even, imagine, must, struggling, ...",...,"[('nightglow, even), (even, imagine), (imagine...","[('nightglow, even, imagine), (even, imagine, ...","[i-imagine-you, you-be-right, you-struggling-r...","